# Portfolio aggregations
In this notebook we'll give some examples on how the portfolio aggregation methods can be used.

### Setting up
We'll set up the imports, data providers, and load the portfolio. For more details about this, please have a look at notebook one (getting started).

In [ ]:
# Only required if you didn't install the module yet. Uncomment the second line and comment out the third line if you wish to use the official release.
# !pip install SBTi
!pip install git+git://github.com/OFBDABV/SBTi

In [1]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario, ScenarioType, EngagementType
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
%aimport -pandas
import pandas as pd

In [2]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/example_portfolio.csv", "example_portfolio.csv")

In [3]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")
df_portfolio = pd.read_csv("example_portfolio.csv", encoding="iso-8859-1")
companies = SBTi.utils.dataframe_to_portfolio(df_portfolio)

In [4]:
temperature_score = TemperatureScore(time_frames=list(SBTi.interfaces.ETimeFrames), scopes=[EScope.S1S2, EScope.S3, EScope.S1S2S3])
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies)

### Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done use different aggregation methods. The termperature scores are calculated per time-frame/scope combination.

In [5]:
def print_aggregations(aggregations):
    aggregations = aggregations.dict()
    for time_frame, time_frame_values in aggregations.items():
        for scope, scope_values in time_frame_values.items():
            print("{} - {}: {} degrees celcius".format(time_frame, scope, scope_values["all"]["score"]))

#### WATS
Weighted Average Temperature Score (WATS): Temperature scores are allocated based on portfolio weights.
This requires the "investment_value" field to be defined in your data set.

In [6]:
temperature_score.aggregation_method = PortfolioAggregationMethod.WATS
print_aggregations(temperature_score.aggregate_scores(amended_portfolio))

short - S1S2: 2.2349090909090905 degrees celcius
short - S3: 2.8205454545454542 degrees celcius
short - S1S2S3: 2.670272727272727 degrees celcius
mid - S1S2: 2.7351818181818177 degrees celcius
mid - S3: 3.0340000000000003 degrees celcius
mid - S1S2S3: 2.9639090909090906 degrees celcius
long - S1S2: 3.122181818181818 degrees celcius
long - S3: 3.2258181818181817 degrees celcius
long - S1S2S3: 3.199636363636363 degrees celcius


#### TETS
Total emissions weighted temperature score (TETS): Temperature scores are allocated based on historical emission weights using total company emissions. 

In [7]:
temperature_score.aggregation_method = PortfolioAggregationMethod.TETS
print_aggregations(temperature_score.aggregate_scores(amended_portfolio))

short - S1S2: 2.29725884873528 degrees celcius
short - S3: 2.4599965951970884 degrees celcius
short - S1S2S3: 2.4295171217466325 degrees celcius
mid - S1S2: 2.8807640760480946 degrees celcius
mid - S3: 2.7215474049114454 degrees celcius
mid - S1S2S3: 2.8232827095737254 degrees celcius
long - S1S2: 2.8746803286943794 degrees celcius
long - S3: 3.2040627788950906 degrees celcius
long - S1S2S3: 3.0651762859922154 degrees celcius


#### MOTS
Market Owned emissions weighted temperature score (MOTS): Temperature scores are allocated based on an equity ownership approach.

In [8]:
temperature_score.aggregation_method = PortfolioAggregationMethod.MOTS
print_aggregations(temperature_score.aggregate_scores(amended_portfolio))

short - S1S2: 2.362759618170431 degrees celcius
short - S3: 2.474672666371289 degrees celcius
short - S1S2S3: 2.4726669521902704 degrees celcius
mid - S1S2: 2.959496104650197 degrees celcius
mid - S3: 2.3070094100485385 degrees celcius
mid - S1S2S3: 2.5932665818284772 degrees celcius
long - S1S2: 3.1149324732273955 degrees celcius
long - S3: 3.202518255484732 degrees celcius
long - S1S2S3: 3.1659784454974766 degrees celcius


#### EOTS
Enterprise Owned emissions weighted temperature score (EOTS): Temperature scores are allocated based
on an enterprise ownership approach. 

In [9]:
temperature_score.aggregation_method = PortfolioAggregationMethod.EOTS
print_aggregations(temperature_score.aggregate_scores(amended_portfolio))

short - S1S2: 2.527586836080241 degrees celcius
short - S3: 2.2113513927400095 degrees celcius
short - S1S2S3: 2.3434074044386035 degrees celcius
mid - S1S2: 2.936485164627035 degrees celcius
mid - S3: 2.000015264219767 degrees celcius
mid - S1S2S3: 2.426945293248226 degrees celcius
long - S1S2: 3.145054144740994 degrees celcius
long - S3: 3.2034859411272287 degrees celcius
long - S1S2S3: 3.179596844185767 degrees celcius


#### ECOTS
Enterprise Value + Cash emissions weighted temperature score (ECOTS): Temperature scores are allocated based on an enterprise value (EV) plus cash & equivalents ownership approach. 

In [10]:
temperature_score.aggregation_method = PortfolioAggregationMethod.ECOTS
print_aggregations(temperature_score.aggregate_scores(amended_portfolio))

short - S1S2: 2.5183561687334626 degrees celcius
short - S3: 2.2988995021956136 degrees celcius
short - S1S2S3: 2.3941135552165145 degrees celcius
mid - S1S2: 2.899183172185718 degrees celcius
mid - S3: 2.2245900661624995 degrees celcius
mid - S1S2S3: 2.54303244408143 degrees celcius
long - S1S2: 3.1281515835381066 degrees celcius
long - S3: 3.203818315296011 degrees celcius
long - S1S2S3: 3.17269502951844 degrees celcius


#### AOTS
Total Assets emissions weighted temperature score (AOTS): Temperature scores are allocated based on a total assets ownership approach. 

In [11]:
temperature_score.aggregation_method = PortfolioAggregationMethod.AOTS
print_aggregations(temperature_score.aggregate_scores(amended_portfolio))

short - S1S2: 1.7406016806601543 degrees celcius
short - S3: 0.9558434722812519 degrees celcius
short - S1S2S3: 1.1726359096429364 degrees celcius
mid - S1S2: 2.075333404902761 degrees celcius
mid - S3: 2.8540947963478565 degrees celcius
mid - S1S2S3: 2.665344839505484 degrees celcius
long - S1S2: 3.185860428469222 degrees celcius
long - S3: 3.2109632177463996 degrees celcius
long - S1S2S3: 3.2032558320539826 degrees celcius
